In [ ]:
import gym
import numpy as np

In [ ]:
#import policy and value iteration

In [ ]:
action_mappings = {
    0: '\u2191', # UP
    1: '\u2192', # RIGHT
    2: '\u2193', # DOWN
    3: '\u2190', # LEFT
}

In [ ]:
# Number of episodes to play
n_episodes = 10000

In [ ]:
def play_episodes(environment, n_episodes, policy):
    wins=0
    total_reward=0
    
    for episode in range(n_episodes):
        state=environment.reset()
        terminated=False
        
        while not terminated:
            #Choose best action according to the policy
            action=np.argmax(policy[state])
            
            #Perform the action in the environment
            next_state, rewards, terminated, info=environment.step(action)
            
            #Add the reward to the total reward count
            total_reward+=rewards
            
            #Update the current state
            state=next_state
            
            #Check if episode is terminated and the reward is achieved....Note that reward is 1 at the goal
            if terminated and rewards==1.0:
                wins+=1
                
    avg_reward=total_reward/n_episodes
    
    return wins, total_reward, avg_reward

__Policy Iteration__

In [ ]:
#Just for fun if you want to check how the environment works, in the below equation 1st value is current state and 2nd value 
#is the action taken. Then environment gives the state_probablity, next_state, reward, terminated
environment.P[50][1]

In [ ]:
#Create a function for policy evaluation. Note this will be done using Bellman Expected Equation
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    #theta: Convergence factor. If the change in value function for all the states is below theta then we are done
    
    print('Evaluating the policy. Max iterations set is:{}'.format(max_iterations))
    
    #Initialise the value function
    V=np.zeros(environment.nS) #Every policy needs to be evaluated from scratch
    
    print('Value function initialised successfully')
    
    #Initialise a counter to monitor the number of evaluations for a policy
    evaluation_iterations=0
    
    for i in range(int(max_iterations)):
        
        #Initialise delta to monitor the convergence
        delta=0
    
    
        for state in range(environment.nS):
            v=0 #This will be used to maintain the value of a state in an iteration

            #Check for all possible actions using the current policy
            for action, action_probablity in enumerate(policy[state]):

                #Now get the next_state probablity, next_state, reward, terminated values from environment
                for state_probablity, next_state, reward, terminated in environment.P[state][action]:

                    #Bellman Expected Equation
                    v+=action_probablity*(reward+(state_probablity*discount_factor*V[next_state]))
                    
            #Capture the change in value of the state Note: We will be maintaining only the maximum delta of all the states
            delta=max(delta,abs(V[state]-v))       

            V[state]=v  #Update the state value function
        
        #increment the counter
        evaluation_iterations+=1
        
               
        #Early stopping
        if(delta<theta):
            print('Policy evaluated in {} iterations'.format(evaluation_iterations))
            print('Final Value function')
            print(V)
            return V  
        

    
                    
            
            

In [ ]:
def cal_q_value(environment, state, state_value_fn, discount_factor=1.0):
    #Here we will be calculating the q-values for given state with every action
    
    #initialise the q-values 
    q_values= np.zeros(environment.nA)
    
    #Note: Q-Value= Reward + DiscountFactor*EnvironmentProbablity*ValueFn
    
    for action in range(environment.nA):
        
        #For each action, environment can take us to any state based on transition model
        for state_probablity, next_state, reward, terminated in environment.P[state][action]:
            
            #Now calculate q-value
            q_values[action]+= reward+(state_probablity*discount_factor*state_value_fn[next_state])
                                       
    return q_values
    

In [ ]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
    
    #Initialise a policy matrix with uniform distribution
    policy=np.ones((environment.nS,environment.nA))/environment.nA
    
    
    #Initialise a variable to store the number of policies evaluated
    evaluated_policies=0
    
    
    
    for i in range(int(max_iterations)):
        
        evaluated_policies+=1  
        
        #Evaluate the current policy
        state_value_fn=policy_evaluation(policy, environment)
        
        print('Value function after {}th policy evaluation:'.format(evaluated_policies))
        print(state_value_fn)
        
        #Initialise a boolean to track the stable policy
        stable_policy=True
                
        #Now we need to act greedily on this evaluated policy. For that we will calculate the q-values for every state
        # and take max. We will calculate the q-values using the evaluated policy for each state
               
        for state in range(environment.nS):
            
            #Get the action using current policy
            current_action=np.argmax(policy[state])
            print('Current policy selects action:{0} for state{1}'.format(current_action, state))
            
            #Calculate the q-values for this state
            q_values= cal_q_value(environment, state, state_value_fn)
            print('Q-values for state:{} is:'.format(state))
            print(q_values)
            
            #Now act greedily to get the best action
            best_action=np.argmax(q_values)
            print('Greedy Policy selects action:{0} for state{1}'.format(best_action, state))
              
            #If the best action is not same as current action, then we need to change the policy
            if (current_action!=best_action):
                stable_policy=False  #Set the policy to False as we need to iterate further
                 
                 
            #Update the policy for the state
            policy[state]=np.eye(environment.nA)[best_action] #This will add the value 1 at the index of best_action
            
        if stable_policy:
            return policy, state_value_fn
            
                
        
        
        
        
    
    

__Lets the create the optimum policy__

In [ ]:
#Create the environment
environment = gym.make('FrozenLake8x8-v0')
print('Environment created.')

In [ ]:
policy, state_value_fn= policy_iteration(environment)

In [ ]:
policy

In [ ]:
state_value_fn

In [ ]:
wins, total_reward, average_reward= play_episodes(environment,1000, policy)

In [ ]:
wins

In [ ]:
total_reward

In [ ]:
average_reward

In [ ]:
policy

In [ ]:
%debug